In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import pandas as pd
import sys
sys.path.append('/home/yat-lok/workspace/tvbtools/')
from tools.signaltools import SignalToolkit
import ipywidgets as widgets
my_layout = widgets.Layout()
import typing
from typing import List
import seaborn as sns

In [ ]:
# read data
filename = '/mnt/d/data/LFP/SNC/2820A/2820A_0.014.csv'
df = pd.read_csv(filename)
df_left = df.iloc[1024:,4]
df_right = df.iloc[1024:,6]
# # visualize results
fig = plt.figure()
axes = fig.add_subplot(211)
axes.plot(df_left)
axes.plot(df_right)
plt.show()


In [ ]:
y = SignalToolkit.sos_filter(df_left, [10,60], 81920)
plt.plot(df_left)
plt.plot(y)
plt.show()

In [ ]:
print(SignalToolkit.PAC(df_right, [2,10], [30,120], fs=81920))
print(SignalToolkit.PAC(df_left, [2,10], [30,120], fs=81920))
# SignalToolkit.PAC_comodulogram(df_right, [2,20,1],[30, 200, 5], fs=81920)
# SignalToolkit.PAC_comodulogram(df_left, [2,20,1],[30, 200, 5], fs=81920)


In [ ]:
# %matplotlib widget
# # hilbert transform
# # step 1, get the amplitude envelop of high freq
# h1=signal.hilbert(df_left_high)
# amplitude_envelope = np.abs(h1)

# # step 2, get the phase information of low freq
# l1 = signal.hilbert(df_left_low)
# phase_y1=np.angle(l1)

# # step 3 bin the phase
# cycle = len(l1)/360
# stamp = range(int(cycle))
# phase_amp = []
# for i in stamp:
#     amp_value = np.mean(amplitude_envelope[i*360:(i+1)*360])
#     phase_amp.append(amp_value)
# # step 4, entropy method H
# p_j = [p_j_single/np.sum(phase_amp) for p_j_single in phase_amp]
# cap_H = -np.sum(p_j * np.log(p_j))

# # step 5, calculate the MI
# MI = (np.log(len(stamp))-cap_H) / np.log(len(stamp))
# print(MI)

# x_axes1 = [*stamp]

# fig = plt.figure()
# axes1 = fig.add_subplot(221)
# axes1.bar(x_axes1, phase_amp)

# # graph 2
# axes2 = fig.add_subplot(222)
# axes2.set_title("raw plot with low and high \nfrequency bands signal")
# axes2.plot(df_left)
# axes2.plot(df_left_low)
# axes2.plot(df_left_high)

# axes3 = fig.add_subplot(223)
# axes3.plot(df_left_high)
# axes3.plot(amplitude_envelope)

# axes4 = fig.add_subplot(224)
# axes4.plot(df_left_low)
# axes4.plot(phase_y1, "o")
# plt.show()